In [19]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt
import PyPDF2 
import tabula
from io import StringIO
from csv import reader
from urllib.request import urlopen
import re

%matplotlib inline

In [20]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
url_list = 'https://olympics.com/en/olympic-games/olympic-results'
r = requests.get(url_list, headers=headers)
soup = bs(r.text, 'html.parser')
games = soup.find_all('section', {'class':'styles__ItemLink-sc-111k07d-2 ipqQzr link-item'})

list_of_games = []
for item in games:
    olympic_games = item.find('p').text
    season =  item.find('span').text
    list_of_games.append((olympic_games, season))


In [21]:
list_of_games = pd.DataFrame({'city':list_of_games})
print(list_of_games)

                                      city
0                   (Beijing 2022, Winter)
1                     (Tokyo 2020, Summer)
2               (PyeongChang 2018, Winter)
3                       (Rio 2016, Summer)
4                     (Sochi 2014, Winter)
..                                     ...
101              (St. Moritz 1948, Winter)
102  (Garmisch-Partenkirchen 1936, Winter)
103             (Lake Placid 1932, Winter)
104              (St. Moritz 1928, Winter)
105                (Chamonix 1924, Winter)

[106 rows x 1 columns]


In [22]:
list_of_games['city'] = list_of_games['city'].astype('string')
winter_games = list_of_games[list_of_games['city'].str.contains('Winter')]
print(winter_games)

                                          city
0                   ('Beijing 2022', 'Winter')
2               ('PyeongChang 2018', 'Winter')
4                     ('Sochi 2014', 'Winter')
6                 ('Vancouver 2010', 'Winter')
8                     ('Turin 2006', 'Winter')
10           ('Salt Lake City 2002', 'Winter')
12                   ('Nagano 1998', 'Winter')
14              ('Lillehammer 1994', 'Winter')
16              ('Albertville 1992', 'Winter')
18                  ('Calgary 1988', 'Winter')
20                 ('Sarajevo 1984', 'Winter')
22              ('Lake Placid 1980', 'Winter')
24                ('Innsbruck 1976', 'Winter')
26                  ('Sapporo 1972', 'Winter')
28                 ('Grenoble 1968', 'Winter')
30                ('Innsbruck 1964', 'Winter')
32             ('Squaw Valley 1960', 'Winter')
34        ("Cortina d'Ampezzo 1956", 'Winter')
36                     ('Oslo 1952', 'Winter')
38               ('St. Moritz 1948', 'Winter')
40   ('Garmis

In [23]:
winter_games = winter_games.replace(to_replace='\(\'', value="", regex=True)
winter_games = winter_games.replace(to_replace=r'\'\)', value="", regex=True)
winter_games = winter_games.replace(to_replace=r'\)', value="", regex=True)
winter_games = winter_games.replace(to_replace=r'\'',value="",regex=True)
winter_games = winter_games.replace(to_replace=r'\,',value="",regex=True)
winter_games = winter_games.replace(to_replace='\(',value="",regex=True)
winter_games = winter_games.replace(to_replace='\"',value="",regex=True)
winter_games

,city
0,Beijing 2022 Winter
2,PyeongChang 2018 Winter
4,Sochi 2014 Winter
6,Vancouver 2010 Winter
8,Turin 2006 Winter
10,Salt Lake City 2002 Winter
12,Nagano 1998 Winter
14,Lillehammer 1994 Winter
16,Albertville 1992 Winter
18,Calgary 1988 Winter


In [24]:
winter_games['season'] = winter_games['city'].apply(lambda x: x.split(' ')[-1])
winter_games['city'] = winter_games.city.str.rsplit(' ',1).str[0]
winter_games

,city,season
0,Beijing 2022,Winter
2,PyeongChang 2018,Winter
4,Sochi 2014,Winter
6,Vancouver 2010,Winter
8,Turin 2006,Winter
10,Salt Lake City 2002,Winter
12,Nagano 1998,Winter
14,Lillehammer 1994,Winter
16,Albertville 1992,Winter
18,Calgary 1988,Winter


In [25]:
winter_games['year'] = winter_games['city'].apply(lambda x: x.split(' ')[-1])
winter_games['city'] = winter_games.city.str.rsplit(' ',1).str[0]
winter_games

,city,season,year
0,Beijing,Winter,2022
2,PyeongChang,Winter,2018
4,Sochi,Winter,2014
6,Vancouver,Winter,2010
8,Turin,Winter,2006
10,Salt Lake City,Winter,2002
12,Nagano,Winter,1998
14,Lillehammer,Winter,1994
16,Albertville,Winter,1992
18,Calgary,Winter,1988


In [26]:
winter_games = winter_games.reset_index()
winter_games

,index,city,season,year
0,0,Beijing,Winter,2022
1,2,PyeongChang,Winter,2018
2,4,Sochi,Winter,2014
3,6,Vancouver,Winter,2010
4,8,Turin,Winter,2006
5,10,Salt Lake City,Winter,2002
6,12,Nagano,Winter,1998
7,14,Lillehammer,Winter,1994
8,16,Albertville,Winter,1992
9,18,Calgary,Winter,1988


In [27]:
# winter_games.drop('index', axis=1, inplace=True)
# code to drop index column is giving error 

In [28]:
winter_games['city'] = winter_games['city'] + '-'
winter_games

,index,city,season,year
0,0,Beijing-,Winter,2022
1,2,PyeongChang-,Winter,2018
2,4,Sochi-,Winter,2014
3,6,Vancouver-,Winter,2010
4,8,Turin-,Winter,2006
5,10,Salt Lake City-,Winter,2002
6,12,Nagano-,Winter,1998
7,14,Lillehammer-,Winter,1994
8,16,Albertville-,Winter,1992
9,18,Calgary-,Winter,1988


In [29]:
winter_games['city_year'] = winter_games['city'] + winter_games['year']
winter_games['city_year'] = winter_games['city_year'].str.lower()
winter_games

,index,city,season,year,city_year
0,0,Beijing-,Winter,2022,beijing-2022
1,2,PyeongChang-,Winter,2018,pyeongchang-2018
2,4,Sochi-,Winter,2014,sochi-2014
3,6,Vancouver-,Winter,2010,vancouver-2010
4,8,Turin-,Winter,2006,turin-2006
5,10,Salt Lake City-,Winter,2002,salt lake city-2002
6,12,Nagano-,Winter,1998,nagano-1998
7,14,Lillehammer-,Winter,1994,lillehammer-1994
8,16,Albertville-,Winter,1992,albertville-1992
9,18,Calgary-,Winter,1988,calgary-1988


In [32]:
winter_games['city_year'] = winter_games['city_year'].str.replace(' ','-').str.replace('.','')

C:\Users\dkham\AppData\Local\Temp/ipykernel_17024/3568862458.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  winter_games['city_year'] = winter_games['city_year'].str.replace(' ','-').str.replace('.','')


In [33]:
winter_games_list = winter_games['city_year'].tolist()

In [34]:
winter_games_list

['beijing-2022',
 'pyeongchang-2018',
 'sochi-2014',
 'vancouver-2010',
 'turin-2006',
 'salt-lake-city-2002',
 'nagano-1998',
 'lillehammer-1994',
 'albertville-1992',
 'calgary-1988',
 'sarajevo-1984',
 'lake-placid-1980',
 'innsbruck-1976',
 'sapporo-1972',
 'grenoble-1968',
 'innsbruck-1964',
 'squaw-valley-1960',
 'cortina-dampezzo-1956',
 'oslo-1952',
 'st-moritz-1948',
 'garmisch-partenkirchen-1936',
 'lake-placid-1932',
 'st-moritz-1928',
 'chamonix-1924',
 'beijing-2022',
 'pyeongchang-2018',
 'sochi-2014',
 'vancouver-2010',
 'turin-2006',
 'salt-lake-city-2002',
 'nagano-1998',
 'lillehammer-1994',
 'albertville-1992',
 'calgary-1988',
 'sarajevo-1984',
 'lake-placid-1980',
 'innsbruck-1976',
 'sapporo-1972',
 'grenoble-1968',
 'innsbruck-1964',
 'squaw-valley-1960',
 'cortina-dampezzo-1956',
 'oslo-1952',
 'st-moritz-1948',
 'garmisch-partenkirchen-1936',
 'lake-placid-1932',
 'st-moritz-1928',
 'chamonix-1924']

In [113]:
# women's individual medalist 

names_list = []
medals_list = []
countries_list = []
winter_games_list2 =[]

for winter_games in winter_games_list:
    '''scraping through olympic url to find women's figure skating results'''
    if winter_games == 'pyeongchang-2018':
        url = f'https://olympics.com/en/olympic-games/{winter_games}/results/figure-skating/ladies-single-skating'
    elif winter_games == 'beijing-2022':
        url = f'https://olympics.com/en/olympic-games/{winter_games}/results/figure-skating/women-single-skating'
    else:
        url = f'https://olympics.com/en/olympic-games/{winter_games}/results/figure-skating/individual-women'
    
    res = requests.get(url)
    soup = bs(res.content)
    
    for row in soup.findAll('div', {'data-cy':'single-athlete-result-row'})[:3]:
        '''pulling the women's names, medals, countries from olympics url'''
        name = row.h3.text
        medal = row.span.text
        country = row.findAll('span',{'data-cy':True})[1].text

        names_list.append(name)
        medals_list.append(medal)
        countries_list.append(country)
        winter_games_list2.append(winter_games)

In [93]:
# women's individual event list
names_list = []
medals_list = []
countries_list = []
winter_games_list2 =[]

winter_games = 'st-moritz-1928'

for row in soup_test.findAll('div', {'data-cy':'single-athlete-result-row'})[:3]:
    name = row.h3.text
    medal = row.span.text
    country = row.findAll('span',{'data-cy':True})[1].text
    
    names_list.append(name)
    medals_list.append(medal)
    countries_list.append(country)
    winter_games_list2.append(games)

In [66]:
url_test = 'https://olympics.com/en/olympic-games/st-moritz-1928/results/figure-skating/individual-women'
response_test = requests.get(url_test)
soup_test = bs(response_test.text)
print(soup_test.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <link href="https://olympics.com/images/static/favicon/apple-touch-icon-16x16.png" rel="apple-touch-icon" sizes="16x16"/>
  <link href="https://olympics.com/images/static/favicon/apple-touch-icon-32x32.png" rel="apple-touch-icon" sizes="32x32"/>
  <link href="https://olympics.com/images/static/favicon/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
  <link href="https://olympics.com/images/static/favicon/apple-touch-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
  <link href="https://olympics.com/images/static/favicon/apple-touch-icon-70x70.png" rel="apple-touch-icon" sizes="70x70"/>
  <link href="https://olympics.com/images/static/favicon/apple-touch-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
  <link href="https://olympics.com/images/static/favicon/apple-touch-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/>
  <link href="https://olympics.com/images/static/favicon/apple-touch-icon-96x96.png" rel="a

In [73]:
for a in soup_test.find_all('a', {'data-cy':'link'}):
    if a.h3: 
        print(a.h3.text)
    

sonja henie
fritzi burger
beatrix loughran
maribel vinson
cecil eustace smith
melitta brunner
ilse hornung
ellen brockhöft
theresa blanchard
andrée brunet
edel randem
kathleen shaw
else flebbe
karen simensen
grete kubitschek
elly winter


In [114]:
games_dict = {
    'name': names_list,
    'medal': medals_list,
    'country': countries_list,
    'games': winter_games_list2
}

In [123]:
women_medals = pd.DataFrame(games_dict)
women_medals
#create dictionary, keys, run .map on country column, df['col'].map(dict)


,name,medal,country,games
0,nathan chen,G,USA,beijing-2022
1,yuma kagiyama,S,JPN,beijing-2022
2,shoma uno,B,JPN,beijing-2022
3,yuzuru hanyu,G,JPN,pyeongchang-2018
4,shoma uno,S,JPN,pyeongchang-2018
...,...,...,...,...
133,willy böckl,S,AUT,st-moritz-1928
134,robert van zeebroeck,B,BEL,st-moritz-1928
135,gillis grafström,G,SWE,chamonix-1924
136,willy böckl,S,AUT,chamonix-1924


## Men's Analysis

In [120]:
# men's individual medalist 

names_list = []
medals_list = []
countries_list = []
winter_games_list2 =[]

for winter_games in winter_games_list:
    '''scraping through olympic url to find men's figure skating results'''
    if winter_games == 'pyeongchang-2018':
        url = f'https://olympics.com/en/olympic-games/{winter_games}/results/figure-skating/mens-single-skating'
    elif winter_games == 'beijing-2022':
        url = f'https://olympics.com/en/olympic-games/{winter_games}/results/figure-skating/men-single-skating'
    else:
        url = f'https://olympics.com/en/olympic-games/{winter_games}/results/figure-skating/individual-men'
    
    res = requests.get(url)
    soup = bs(res.content)
    
    for row in soup.findAll('div', {'data-cy':'single-athlete-result-row'})[:3]:
        '''pulling the men's names, medals, countries from olympics url'''
        name = row.h3.text
        medal = row.span.text
        country = row.findAll('span',{'data-cy':True})[1].text

        names_list.append(name)
        medals_list.append(medal)
        countries_list.append(country)
        winter_games_list2.append(winter_games)

In [122]:
games_dict = {
    'name': names_list,
    'medal': medals_list,
    'country': countries_list,
    'games': winter_games_list2
}

men_medals = pd.DataFrame(games_dict)
men_medals

,name,medal,country,games
0,nathan chen,G,USA,beijing-2022
1,yuma kagiyama,S,JPN,beijing-2022
2,shoma uno,B,JPN,beijing-2022
3,yuzuru hanyu,G,JPN,pyeongchang-2018
4,shoma uno,S,JPN,pyeongchang-2018
...,...,...,...,...
133,willy böckl,S,AUT,st-moritz-1928
134,robert van zeebroeck,B,BEL,st-moritz-1928
135,gillis grafström,G,SWE,chamonix-1924
136,willy böckl,S,AUT,chamonix-1924
